In [ ]:
from matplotlib import pyplot as plt
from scipy.optimize import newton
from scipy.integrate import quad

import numpy as np

In [ ]:
g = 9.81
length = 0.6
height = 0.4

In [ ]:
def cycloid(final_x, final_y):
    theta2 = newton(
        lambda angle: final_y / final_x - (1 - np.cos(angle)) / (angle - np.sin(angle)),
        np.pi / 2
    )

    R = final_y / (1 - np.cos(theta2))
    theta = np.linspace(0, theta2, 1000)

    return R * (theta - np.sin(theta)), R * (1 - np.cos(theta)), theta2 * np.sqrt(R / g)

In [ ]:
def linear(final_x, final_y):
    m = final_y / final_x
    x = np.linspace(0, final_x, 1000)

    return x, m * x, np.sqrt((1 + m ** 2) / (10 / 7 * g * m * final_x))

In [ ]:
def func(x, f, fp):
    return np.sqrt((1 + fp(x) ** 2) / (10 / 7 * g * f(x)))


def quadratic(final_x, final_y):
    def f(x):
        return final_y / final_x ** (1 / 2) * x ** (1 / 2)

    def fp(x):
        return (1 / 2) * final_y / final_x ** (1 / 2) * x ** (1 / 2 - 1)

    x = np.linspace(0, final_x, 1000)
    y = f(x)

    return x, y, quad(func, 0, final_x, args=(f, fp))[0]


def interpolate(final_x, final_y):
    x, y, T = cycloid(final_x, final_y)

    coef = np.polyfit(x, y, deg=5)
    coef_reversed = coef[::-1]

    def f(x):
        return sum([coef_reversed[i] * x ** i for i in range(len(coef))])

    def fp(x):
        return sum([i * coef_reversed[i] * x ** (i - 1) for i in range(len(coef))])

    T = quad(func, 0, final_x, args=(f, fp))[0]

    return [x, np.polyval(coef, x), T]

In [ ]:
fig, ax = plt.subplots()

for curve, function in [('cycloid', 'cycloid'), ('linear', 'linear'), ('quadratic', 'quadratic')]:
    x, y, T = globals()[function](length, height)
    ax.plot(x, y, lw=2, alpha=0.5, label=f'{curve}: {round(T * 1000)} ms')

x, y, T = interpolate(length, height)
ax.plot(x, y, label=f'interpolation: {round(T * 1000)} мс')

ax.legend()
ax.set_xlim(0, length)
ax.set_ylim(height, 0)

plt.show()